# Matching companies across dataset 

This Jupyter Notebook contains code on creating a dataframe that demonstrates all the company matches between:
 * FDA and NDC
 * FDA and DNA
 * NDC and DNA 
 * NDC and FDA Matrix
 
The function that creates the table output for corporate family and all matches in the second df is defined here.

In [9]:
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz
#from nltk.metrics import jaccard_distance
from numpy import save
import itertools

### Function for matching and matrix

In [13]:
##Function that takes in 3 lists, the list of strings you want to compare, the list of strings you're comparing to and a list with the original way the string from list 1 was written.  
def matchWithFuzzyNamesAcross(l1, l2, og1, og2, row_num1, row_num2):
    
    """outputs a dataframe with the clean company name, original company name and original row for all the 100% matches
    
        Keyword Arguments:
        l1 -- a column or list of the clean company names being compared
        l2 -- a column or list of the clean company you're comparing to
        og1 -- a column or list of the original  company names being compared
        og2 -- a column or list of the original  company names you're comparing to
        row_num1 -- a column or list of the original row numbers (before removing NA values) being compared 
        row_num2 -- a column or list of the original row numbers (before removing NA values) you're comparing to 
    """
    match = pd.DataFrame(columns = ['clean fda company name', 'original fda company', 'fda row', 'clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio'])
    
    for (i, o, r) in zip(l1, og1, row_num1):
        
        '''
        this will go through each row in all three lists at the same time:
        for each clean company name (i) , original company name (o) and original row number (r) 
        in each row of clean_company, original_company and original row number list being compared 

        '''
        
        #these lists are defined in this first for loop so the list becomes empty and can store new information for each company name being matched 
        
        matches = [] #empty list to store all the matches for current row 
        
        score = [] #empty list to store all the scores for current row
        
        #score_partial = []
        
        row = [] #empty list to store all the original row numbers for current row 
        
        matchesog = [] #empty list to store all original company names
        
        for (j, q, s) in zip(l2, og2, row_num2):
            '''
            this will go through each row in all three lists at the same time:
            for each clean company name (j) , original company name (q) and original row number (s) 
            in each row of clean_company, original_company and original row number of the list you're comparing to
        
            '''
            
    
            if (fuzz.ratio(i,j)) == 100: #due to time constraints, we set the fuzz.ratio to 100, however future programming should adjust this value to some other quantity to see if we get better matches
                
                matches.append(j) #add all the clean company names from list 2 that matches to clean company name from list 1 to the empty matches list 
               
                matchesog.append(q) #add all the original company names associated with the clean company name in list 2 to the empty matches og list
                
                score.append(fuzz.ratio(i,j)) #add all the fuzz ratios associated with comparing clean company in list 1 to clean company in list 2 to the list score
                
                row.append(s) #add all the original row number associated with the clean company name in list 2 to the row list 
                
                #the partial ratio will return the ratio raw score of 100  if the shorter string is found an any of substrings of the larger string, used this as a safety net and potentially as an additional step to 
                #score_partial.append((fuzz.partial_ratio(i,j)))
            
            
                #match is defined within this if statement because we only want the clean fda company that matches this criteria of 100% match
                match = match.append({ 'clean fda company name': i, 'original fda company': o, 'fda row': r, 'clean ndc row': row, 'clean ndc company': matches,   'original ndc company': matchesog, 'fuzz ratio': score,},ignore_index=True)
        
#Will output a pandas dataframe with 4 columns
    return match


In [ ]:
#OLD CODE IDEA: First try to incorporate pandas and numpy; the final output should be a pandas dataframe that is a matrix. If the code is still too slow, then break up the dna dataset
#into three 20000x60000 have three seperate jupyters running, using this function for each third of the data(dna), and get 
#put write.csv into function(will have three csvs, which will need to combined)

#try to get a frequency distribution of the matches

#NEW CODE:
def matrix_score(l1, l2, file_name):
    """outputs a numpy array with all the fuzz ratio for each pairing of words between l1 and l2
    
        Keyword Arguments:
        l1 - a column or list of the clean company names being compared
        l2 - a column or list of the clean company you're comparing to
        file_name - the filepath file will be saved to 

    """
    #creates a matrix of zeros with the dimensions specified from the parameters
    #So if you had a list that had 5 elements in it, your matrix would be a 5x5 matrix; nrows would be 5 and ncols would be 5
    #matrix = np.zeros(shape = (nrows, ncols))
    matrix = np.zeros(shape = (len(l1), len(l2)))
    
    #initialize row and col values to 0
    row = 0
    col = 0
    
    for i in l1:
        if row % (len(l1)//10) == 0:       
            print("progress: %.2f" % (100*row/len(l1)) + "%") #prints out the progress of, will update approximately every 10%
        for j in l2:
            score = fuzz.ratio(i,j)
            matrix[row][col] = score
            col+=1
        col = 0
        row+=1
    
    save(file_name, matrix) #uses numpy package to automatically save array as an npy file 
    
    return matrix 

### Setting up inputs for FDA x NDC

In [10]:
ndc = pd.read_csv("../../data/working/ndc_clean.csv", index_col = [0])

In [11]:
ndc.dropna(inplace = True) #drop NA values, NA values produced due to aggresive cleaning methods (check cleaning folder)
ndc.reset_index(inplace = True) #reset the index to match the new number of rows without NA values 

In [12]:
ndc_row = ndc['row'] #this input is needed to match the original row number (the row number of df with NA value) where the matches comes from 

In [13]:
ndc_og = ndc['original_company'] #this input is needed to list the original company name, pre-cleaning

In [14]:
ndc_clean = ndc['cleaned_name'] #this input is needed as it is what we used for matching 

In [15]:
fda = pd.read_csv("../data/working/fda_clean.csv", index_col = [0])

In [16]:
fda.dropna(inplace = True) #drop NA values, NA values produced due to aggresive cleaning methods (check cleaning folder)
fda.reset_index(inplace = True)#reset the index to match the new number of rows without NA values 

In [9]:
fda.columns

Index(['index', 'FDA Companies ', 'Company Clean'], dtype='object')

In [10]:
fda_row = fda['index'] #this input is needed to match the original row number (the row number of df with NA value) where the matches comes from 

In [11]:
fda_og = fda['FDA Companies '] #this input is needed to list the original company name, pre-cleaning let's us see what company names are truly matching

In [12]:
fda_clean = fda['Company Clean'] #this input is needed as it is what we used for matching 

#### FDAxNDC Matching

In [14]:
fdaxndc = matchWithFuzzyNamesAcross(fda_clean, ndc_clean, fda_og, ndc_og, fda_row, ndc_row)

In [64]:
fdaxndc

,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,m,3M,1,"[20, 22]","[m, m]","[3M Company, 3M Health Care]","[100, 100]"
2,abbvie,ABBVIE,5,[66],[abbvie],[AbbVie Inc.],[100]
3,abbvie,ABBVIE INC,8,[66],[abbvie],[AbbVie Inc.],[100]
4,abraxis bioscience,ABRAXIS BIOSCIENCE,11,[71],[abraxis bioscience],"[Abraxis BioScience, LLC]",[100]
5,accord,ACCORD HLTHCARE,12,"[81, 82, 83, 84, 85]","[accord, accord, accord, accord, accord]","[Accord Healthcare, Accord Healthcare Inc, Acc...","[100, 100, 100, 100, 100]"
...,...,...,...,...,...,...,...
913,yaopharma,YAOPHARMA CO LTD,966,[6939],[yaopharma],"[Yaopharma Co., Ltd.]",[100]
914,zo skin,ZO SKIN HEALTH,970,[7009],[zo skin],"[ZO Skin Health, Inc.]",[100]
915,zydus,ZYDUS HLTHCARE,971,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"
919,zydus,ZYDUS PHARMS USA,972,"[7017, 7018, 7019, 7020]","[zydus, zydus, zydus, zydus]","[Zydus Pharmaceuticals (USA) Inc., Zydus Pharm...","[100, 100, 100, 100]"


### Cleaning up the results to obtain Table with unique row #'s

In [15]:
#The code produces repeats the same output for each match a string has, (ex. if car matched 4x it would have the same four entries for car) this code removes duplicate rows leaving only the one match we're interested in
fdaxndc = fdaxndc[~fdaxndc.duplicated('fda row')]

In [59]:
#failed attempts
#ndc_unique = fdaxndc[~fdaxndc.duplicated('clean ndc company')]
#ndc_unique = fdaxndc.drop_duplicates(subset='clean ndc company')

TypeError: unhashable type: 'list'

In [16]:
#changing the 3 columns we want to aggregate into a single row into list to facilitate joining of information
fdaxndc['clean fda company name'] = fdaxndc['clean fda company name'].astype(str)
fdaxndc['original fda company'] = fdaxndc['original fda company'].astype(str)
fdaxndc['fda row'] = fdaxndc['fda row'].astype(str)

In [17]:
#recent change output will not reflect -- changing joining character from comma to carrot in order to split more easily
#groups by clean fda company and joins all the ways the clean fda company is originally written along with all the rows where these original companies are found in the original dataframe
fdaxndc_unique = fdaxndc.groupby('clean fda company name', as_index = False).agg({'original fda company': '^ '.join, 
                                                                            'fda row': '^ '.join
                                                                                })

In [18]:
#groups by clean fda company name and only keeps the first instance of all the rows (non-repeats)
fdaxndc_unique1 = fdaxndc.groupby('clean fda company name', as_index = False).first()

In [22]:
#This creates the final data frame output that has one clean corporate family and all the matches in from the other company dataset in the same row. 
fdaxndc_unique[['corporate family', 'clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio']] = fdaxndc_unique1[['clean fda company name','clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio']]

In [ ]:
#fdaxndc_unique.to_csv('../data/working/fda_ndc_matching.csv')

## FDA x DNA

### Preparing DNA inputs 

In [24]:
#read in the dna dataset 
dna = pd.read_csv("../../data/working/dna_clean.csv", index_col = [0])

In [25]:
dna.dropna(inplace = True) #drop NA values, NA values produced due to aggresive cleaning methods (check cleaning folder)
dna.reset_index(inplace = True) #reset the index to match the new number of rows without NA values

In [28]:
dna_row = dna['index']
dna_og = dna['Description']
dna_clean = dna['cleaned_companies']

### Using matchWithFuzzyNamesAcross to create dateframe with matches

In [29]:
fdaxnda = matchWithFuzzyNamesAcross(fda_clean, dna_clean, fda_og, dna_og, fda_row, dna_row)

In [31]:
fdaxnda.to_csv('DNA and FDA matches') #last minute save because session was about to end 

In [24]:
#THIS IS OLD CODE NO LONG RELEVANT
fdaxnda= pd.read_csv('DNA and FDA matches') #reading in the last minute df save to finish output

In [3]:
fdaxnda

,Unnamed: 0,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,0,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
1,1,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
2,2,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
3,3,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
4,4,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
...,...,...,...,...,...,...,...,...
613,613,wockhardt,WOCKHARDT LTD,953,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
614,614,wockhardt,WOCKHARDT USA,954,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
615,615,wyeth,WYETH PHARMS INC,957,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"
616,616,wyeth,WYETH PHARMS INC,957,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"


### Cleaning up dataframe to only have one corporate family and all atches in one row 

In [25]:
fdaxnda = fdaxnda[~fdaxnda.duplicated('fda row')] #remove any duplicate of row numbers from list being compared, only want 1 input from the list being compared to 

In [26]:
fdaxnda

,Unnamed: 0,clean fda company name,original fda company,fda row,clean ndc row,clean ndc company,original ndc company,fuzz ratio
0,0,m,3M,1,"[33223, 37340, 38604, 53031, 56261, 58045]","['m', 'm', 'm', 'm', 'm', 'm']","['M101 Corp', '3M Company', 'M2 Group Ltd.', '...","[100, 100, 100, 100, 100, 100]"
6,6,aaipharma,AAIPHARMA LLC,3,[3820],['aaipharma'],['AAIPharma Services Corp.'],[100]
7,7,abbott,ABBOTT LABS,4,"[260, 273, 275]","['abbott', 'abbott', 'abbott']","['Abbott AG', 'Abbott Oy', 'Abbott Srl']","[100, 100, 100]"
10,10,abbvie,ABBVIE,5,[488],['abbvie'],['AbbVie Inc.'],[100]
11,11,abbvie,ABBVIE INC,8,[488],['abbvie'],['AbbVie Inc.'],[100]
...,...,...,...,...,...,...,...,...
612,612,wockhardt bio,WOCKHARDT BIO AG,952,[62283],['wockhardt bio'],['Wockhardt Bio AG'],[100]
613,613,wockhardt,WOCKHARDT LTD,953,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
614,614,wockhardt,WOCKHARDT USA,954,[61416],['wockhardt'],['Wockhardt Ltd'],[100]
615,615,wyeth,WYETH PHARMS INC,957,"[1808, 15691]","['wyeth', 'wyeth']","['Wyeth LLC', 'Wyeth Ltd.']","[100, 100]"


In [27]:
#changing the 3 columns we want to aggregate into a single row into list to facilitate joining of information
fdaxnda['clean fda company name'] = fdaxnda['clean fda company name'].astype(str)
fdaxnda['original fda company'] = fdaxnda['original fda company'].astype(str)
fdaxnda['fda row'] = fdaxnda['fda row'].astype(str)

In [28]:
#recent change output will not reflect -- changing joining character from comma to carrot in order to split more easily
#groups by clean fda company and joins all the ways the clean fda company is originally written along with all the rows where these original companies are found in the original dataframe
fdaxnda_unique = fdaxnda.groupby('clean fda company name', as_index = False).agg({'original fda company': '^ '.join, 
                                                                            'fda row': '^ '.join
                                                                                })

In [29]:
#groups by clean fda company name and keeps the first instance of all the rows (non-repeats) allows us to have a second df with all the information we want
fdaxnda1 = fdaxnda.groupby('clean fda company name', as_index = False).first()

In [31]:
#matches the first df columns with the necessary information from the second df to complete the final output
fdaxnda_unique[['corporate family', 'clean dna row', 'clean dna company', 'original dna company', 'fuzz ratio']] = fdaxnda1[['clean fda company name','clean ndc row', 'clean ndc company', 'original ndc company', 'fuzz ratio']]

In [33]:
#fdaxnda_unique.to_csv('../data/working/fda_dna_matching.csv')

### NDCxDNA

In [ ]:
#save the output of function to variable ndcxdna_match
ndcxdna= matchWithFuzzyNamesAcross(ndc_clean, dna_clean, ndc_og, dna_og, ndc_row, dna_row)

In [ ]:
#changing the 3 columns we want to aggregate into a single row into list to facilitate joining of information
ndcxdna= ndcxdna_match[~ndcxdna_match.duplicated('NDC row')]

In [ ]:
#recent change output will not reflect -- changing joining character from comma to carrot in order to split more easily
#groups by clean fda company and joins all the ways the clean fda company is originally written along with all the rows where these original companies are found in the original dataframe
ndcxdna_match['clean NDC company'] = ndcxdna_match['clean NDC company'].astype(str)
ndcxdna_match['original NDC company'] = ndcxdna_match['original NDC company'].astype(str)
ndcxdna_match['NDC row'] = ndcxdna_match['NDC row'].astype(str)

In [ ]:
ndcxdna_unique = ndcxdna_match.groupby('clean NDC company', as_index = False).agg({'original NDC company': '^ '.join, 
                                                                            'NDC row': '^ '.join
                                                                                })

In [ ]:
#groups by clean fda company name and keeps the first instance of all the rows (non-repeats) allows us to have a second df with all the information on clean DNA row, clean DNA company, original DNA company and fuzz ratio
ndcxdna_1 = ndcxdna_match.groupby('clean NDC company', as_index = False).first()

In [ ]:
#matches the first df columns with the necessary information from the second df to complete the final output
ndcxdna_unique[['corporate family', 'clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio']] = ndcxdna_match1[['clean NDC company','clean DNA row', 'clean DNA company', 'original DNA company', 'fuzz ratio']]

### Matrix FDAxNDC

In [ ]:
#Getting unique clean companies from ndc_matching (see ndc matrix section above to see where this variable came from)
ndc_unique_clean = ndcxdna_unique['clean NDC company']
#drops any duplicates so that the result is a series of unique companies that matched 100%
ndc_unique_clean.drop_duplicates(inplace = True)
#convert series to a df
ndc_unique_clean = pd.DataFrame(ndc_unique_clean)
#renaming column
ndc_unique_clean = ndc_unique_clean.rename(columns = {"clean company name": "ndc clean name"})
#reset index so numbers are in order
ndc_unique_clean.reset_index(inplace = True)
#delete index column
del ndc_unique_clean['index']

In [ ]:
#Getting unique clean companies from fda (see fda matrix section above to see where this variable came from)
fda_unique_clean = fdaxdna['clean fda company name']
fda_unique_clean.drop_duplicates(inplace = True)
fda_unique_clean = pd.DataFrame(fda_unique_clean)
fda_unique_clean = fda_unique_clean.rename(columns = {"clean company name": "fda clean name"})
fda_unique_clean.reset_index(inplace = True)
del fda_unique_clean['index']

In [ ]:
#creates a 796 x 5209 matrix
fdaxndc_matrix = matrix_score(fda_unique_clean['fda clean name'].tolist(), ndc_unique_clean['ndc clean name'].tolist())

In [ ]:
#ndcxdna_match_unique.to_csv('../data/working/ndc_dna_matching.csv')